In [15]:
"""
Generate enhanced dataset + train & export model.

Outputs:
- towers_metadata.csv         (static per tower: region, age, corrosion, coords, distance, last_maintenance)
- tower_daily.csv             (per-tower-per-day dynamic sensor values + labels)
- graph_edges.csv             (sparse undirected road graph incl. substation edges)
- tower_data.csv              (merged training-ready table, for convenience)
- fault_prediction_model.pkl  (scikit-learn pipeline)
- model_features.pkl          (ordered feature list for inference alignment)
- roc_curve.png, confusion_matrix.png (saved plots)
"""

import numpy as np
import pandas as pd
import random
from datetime import datetime, timedelta
from itertools import combinations
import os

# ML
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score,
    precision_score, recall_score, roc_auc_score, roc_curve
)
import matplotlib.pyplot as plt
import joblib

# -------------------
# Reproducibility
# -------------------
np.random.seed(42)
random.seed(42)

# -------------------
# Parameters
# -------------------
n_towers = 25
n_days = 60                 # ~2 months
start_date = datetime(2024, 5, 1)
regions = ["North", "South", "East", "West"]

# Coordinate space and substation
GRID_MIN, GRID_MAX = 0, 100
SUB_ID = "SUB"
SUB_X, SUB_Y = 50, 50

# Road graph sparsity
K_NEIGHBORS = 4          # each tower connects to its 4 nearest towers
K_SUB_EDGES = 6          # substation connects to its 6 nearest towers

# -------------------
# Helper functions
# -------------------
def euclidean(x1, y1, x2, y2):
    return float(np.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2))

def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

# -------------------
# Generate static tower metadata (one row per tower)
# -------------------
towers = [f"T{i+1}" for i in range(n_towers)]
meta_rows = []

for t in towers:
    tower_age = np.random.randint(5, 40)
    corrosion = np.random.randint(0, 100)
    region = random.choice(regions)
    region_risk = np.random.randint(1, 10)
    last_maintenance = datetime(2024, 1, 1) + timedelta(days=np.random.randint(0, 365))

    # coordinates
    x = np.random.randint(GRID_MIN, GRID_MAX + 1)
    y = np.random.randint(GRID_MIN, GRID_MAX + 1)
    dist_sub = euclidean(x, y, SUB_X, SUB_Y)

    meta_rows.append({
        "tower_id": t,
        "region": region,
        "region_risk_score": region_risk,
        "tower_age_years": tower_age,
        "corrosion_level": corrosion,
        "last_maintenance": last_maintenance.date(),
        "x": x,
        "y": y,
        "distance_from_substation": dist_sub
    })

df_meta = pd.DataFrame(meta_rows)
df_meta.to_csv("towers_metadata.csv", index=False)

# -------------------
# Build a sparse undirected road graph (edges)
# -------------------
# compute pairwise distances
coords = df_meta.set_index("tower_id")[["x", "y"]].to_dict("index")

# K-nearest neighbors per tower
edges = set()
for t in towers:
    x1, y1 = coords[t]["x"], coords[t]["y"]
    dists = []
    for u in towers:
        if u == t: 
            continue
        x2, y2 = coords[u]["x"], coords[u]["y"]
        d = euclidean(x1, y1, x2, y2)
        dists.append((u, d))
    dists.sort(key=lambda z: z[1])
    for u, d in dists[:K_NEIGHBORS]:
        edge = tuple(sorted([t, u]))
        edges.add((edge[0], edge[1], d))

# connect substation to K nearest towers
df_meta_sorted = df_meta.sort_values("distance_from_substation")
for _, row in df_meta_sorted.head(K_SUB_EDGES).iterrows():
    d = row["distance_from_substation"]
    edges.add((SUB_ID, row["tower_id"], d))

df_edges = pd.DataFrame(list(edges), columns=["source", "target", "distance"])
df_edges.to_csv("graph_edges.csv", index=False)

# -------------------
# Generate daily dynamic data and labels
# -------------------
daily_rows = []
for t in towers:
    meta = df_meta[df_meta["tower_id"] == t].iloc[0]
    tower_age = meta["tower_age_years"]
    corrosion = meta["corrosion_level"]
    region_risk = meta["region_risk_score"]
    last_maint = pd.to_datetime(meta["last_maintenance"])

    for day in range(n_days):
        date = start_date + timedelta(days=day)
        # dynamic sensors
        rain = np.random.randint(0, 100)
        wind = np.random.randint(0, 120)
        temp = np.random.randint(-5, 45)
        freq = np.random.uniform(49, 51)
        volt = np.random.uniform(200, 240)
        overload = np.random.randint(50, 150)

        # anomalies
        freq_anom = 1 if (freq < 49.5 or freq > 50.5) else 0
        volt_anom = 1 if (volt < 210 or volt > 230) else 0
        anomaly_chances = freq_anom + volt_anom

        # days since maintenance (semi-dynamic)
        days_since_maintenance = (date - last_maint).days

        # underlying risk score (same logic you used + small noise)
        score = (
            0.4 * anomaly_chances +
            0.3 * (corrosion / 100) +
            0.2 * (tower_age / 40) +
            0.3 * (rain / 100) +
            0.2 * (wind / 120) +
            0.1 * (region_risk / 10) +
            np.random.normal(0, 0.05)
        )
        prob_fault = sigmoid(score)
        faulty = np.random.rand() < prob_fault

        daily_rows.append({
            "tower_id": t,
            "date": date.date(),
            "rain_intensity": rain,
            "wind_speed": wind,
            "temperature": temp,
            "frequency": freq,
            "voltage": volt,
            "overload_current_pct": overload,
            "freq_anomaly": freq_anom,
            "volt_anomaly": volt_anom,
            "anomaly_chances": anomaly_chances,
            "days_since_maintenance": days_since_maintenance,
            "faulty": int(faulty)
        })

df_daily = pd.DataFrame(daily_rows)
df_daily.to_csv("tower_daily.csv", index=False)

# -------------------
# Merge training table (static + dynamic)
# -------------------
df_train = df_daily.merge(df_meta, on="tower_id", how="left")

# For convenience, also save a single training csv like before
df_train.to_csv("tower_data.csv", index=False)

# -------------------
# Train / Evaluate / Export model (Logistic Regression pipeline)
# -------------------
# Features & target
drop_cols = ["date", "last_maintenance", "faulty", "tower_id"]
X = df_train.drop(columns=drop_cols)
# one-hot region
X = pd.get_dummies(X, columns=["region"], drop_first=True)
y = df_train["faulty"]

# split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=2000, random_state=42))
])

param_grid = {
    "clf__C": [0.1, 1, 10,100],
    "clf__penalty": ["l1", "l2"],
    "clf__solver": ["liblinear", "saga"]
}

grid = GridSearchCV(
    pipeline, param_grid, cv=5, scoring="f1", n_jobs=-1, verbose=0
)
grid.fit(X_train, y_train)

best_model = grid.best_estimator_

# predictions & metrics
y_pred = best_model.predict(X_test)
y_prob = best_model.predict_proba(X_test)[:, 1]

print("Best Params:", grid.best_params_)
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_prob))

# plots saved to disk
# ROC
fpr, tpr, _ = roc_curve(y_test, y_prob)
plt.figure()
plt.plot(fpr, tpr)
plt.plot([0,1],[0,1],'--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.tight_layout()
plt.savefig("roc_curve.png", dpi=160)
plt.close()

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure()
plt.imshow(cm, interpolation="nearest")
plt.title("Confusion Matrix")
plt.colorbar()
tick_marks = np.arange(2)
plt.xticks(tick_marks, ["No Fault", "Fault"])
plt.yticks(tick_marks, ["No Fault", "Fault"])
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, cm[i, j], ha="center", va="center")
plt.ylabel("True label")
plt.xlabel("Predicted label")
plt.tight_layout()
plt.savefig("confusion_matrix.png", dpi=160)
plt.close()

# Export model + feature order (critical for inference)
joblib.dump(best_model, "fault_prediction_model.pkl")
joblib.dump(list(X.columns), "model_features.pkl")

print("\nFiles written:")
for f in [
    "towers_metadata.csv", "tower_daily.csv", "graph_edges.csv",
    "tower_data.csv", "fault_prediction_model.pkl", "model_features.pkl",
    "roc_curve.png", "confusion_matrix.png"
]:
    print(" -", os.path.abspath(f))


Best Params: {'clf__C': 0.1, 'clf__penalty': 'l1', 'clf__solver': 'liblinear'}

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        79
           1       0.74      1.00      0.85       221

    accuracy                           0.74       300
   macro avg       0.37      0.50      0.42       300
weighted avg       0.54      0.74      0.62       300

Accuracy: 0.7366666666666667
Precision: 0.7366666666666667
Recall: 1.0
ROC-AUC: 0.5988315481986368


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Files written:
 - C:\Users\Prisha\Desktop\POWERGRID\towers_metadata.csv
 - C:\Users\Prisha\Desktop\POWERGRID\tower_daily.csv
 - C:\Users\Prisha\Desktop\POWERGRID\graph_edges.csv
 - C:\Users\Prisha\Desktop\POWERGRID\tower_data.csv
 - C:\Users\Prisha\Desktop\POWERGRID\fault_prediction_model.pkl
 - C:\Users\Prisha\Desktop\POWERGRID\model_features.pkl
 - C:\Users\Prisha\Desktop\POWERGRID\roc_curve.png
 - C:\Users\Prisha\Desktop\POWERGRID\confusion_matrix.png
